<a href="https://colab.research.google.com/github/Prathamesh-Chavan-98/AI-Interviewer/blob/main/AI_Interviewer_gr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain community langchain chroma
!pip install openai
!pip install getpass
!pip install groq
!pip install pypdf
!pip install pymupdf
!pip install python-dotenv
!pip install chromadb

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


In [ ]:
import os
!pip install langchain_community
from getpass import getpass
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
import fitz
from langchain.document_loaders import TextLoader
import fitz
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain import hub
from google.colab import userdata
from groq import Groq
from dotenv import load_dotenv

In [ ]:
groq_api_key = userdata.get("groq-api-key")
langchain_api_key = userdata.get("langchain-api-key")

# connecting with llm (llama-3.3-70b-versatile)

In [ ]:
from dotenv import load_dotenv
GROQ_API_KEY=groq_api_key

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
api_key = groq_api_key

# Check if the API key is available
if not api_key:
    raise ValueError("API key not found. Set GROQ_API_KEY in the .env file or environment variables.")

# Initialize the Groq client
client = Groq(api_key=api_key)

# Send a chat completion request
try:
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "hii i am Prathamesh Chavan",
            }
        ],
        model="llama-3.3-70b-versatile",
    )

    # Print the response
    print(chat_completion.choices[0].message.content)

except Exception as e:
    print(f"An error occurred: {e}")


Hello Prathamesh Chavan, it's nice to meet you. Is there something I can help you with or would you like to chat?


# Load the PDF document and Chunking


In [ ]:
# Load the PDF document
loader = PyPDFLoader("/content/PRC Resume.pdf")
docs = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# Creating Embeddings

In [ ]:

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create a vector store (Chroma) from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Create a retriever
retriever = vectorstore.as_retriever()

# Load a pre-built RAG prompt
prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# Docs Formatting

In [ ]:
# Helper function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Function to generate a response using RAG
def generate_rag_response(question):
    # Retrieve relevant documents
    relevant_docs = retriever.get_relevant_documents(question)

    if not relevant_docs:
        return "No relevant documents found."

    # Format the retrieved documents as context
    context = format_docs(relevant_docs)

    # Fill the prompt with retrieved context and the question
    formatted_prompt = prompt.format(context=context, question=question)

    # Get the response from the LLM
    response = get_groq_response(formatted_prompt)

    return response


# Using groq for all LLM related tasks

In [ ]:
def get_groq_response(prompt, llm_name="llama-3.3-70b-versatile"):
    """
    Gets a response from the Groq API.

    Args:
        prompt: The prompt to send to the API.
        llm_name: The name of the LLM to use.

    Returns:
        The response from the API.
    """
    # Access the groq_api_key from the global scope
    global groq_api_key

    client = Groq(
        api_key=groq_api_key,
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=llm_name,
    )

    return chat_completion.choices[0].message.content

# User skills & AOI input

In [ ]:
# Collect user input about their skills and area of interest
user_skills = input("Enter your skills (comma-separated): ").strip().split(',')
user_area_of_interest = input("Enter your area of interest: ").strip()

print(f"\nSkills: {user_skills}")
print(f"Area of Interest: {user_area_of_interest}")


Enter your skills (comma-separated): data science 
Enter your area of interest: data science

Skills: ['data science']
Area of Interest: data science


# Generation of Questions

In [ ]:
# Combine all input data into a query prompt
def generate_relevant_questions():
    question_prompt = f"""
    Generate 10 relevant and very perticular moderate level interview questions for a candidate based on the following information generate 4 questions based on resume and project and skills in resume 3 on area of interest and 3 on skills of candidate:
    - Resume Content: {splits}
    - Skills: {user_skills}
    - Area of Interest: {user_area_of_interest}
    """

    # Get the response and extract content as string
    response = generate_rag_response(question_prompt)
    return response  # Access the content attribute of AIMessage

# Generate and display questions
questions = generate_relevant_questions()

# Print the generated questions
print("\nTop 10 Relevant Questions:")
for idx, question in enumerate(questions.split('\n'), 1):
    print(f"{idx}. {question}")


Top 10 Relevant Questions:
1. Here are 10 interview questions for the candidate: 
2. 4 based on resume and project: 
3. 1. How did you engineer the navigation system for Hydrominex, and what technologies did you use?
4. 2. Can you explain your experience with machine learning software for real-time weather forecasting and alerts?
5. 3. How did you optimize retrieval accuracy in your AI Interviewer project to handle complex queries?
6. 4. What was your role in building the automated news summarization and analysis tool, and what technologies did you use?
7. 3 based on area of interest (data science): 
8. 5. What do you think are the most significant challenges in the field of data science, and how do you propose to address them?
9. 6. Can you describe a project where you applied data science techniques to solve a real-world problem?
10. 7. How do you stay updated with the latest developments and advancements in the field of data science?
11. 3 based on skills (data science): 
12. 8. Ho

# Accepting Answers

In [ ]:
# Function to dynamically generate questions using RAG
def generate_relevant_questions():
    question_prompt = f"""
    Generate 10 relevant and very perticular moderate level interview questions for a candidate based on the following information generate 4 questions based on resume and project and skills in resume 3 on area of interest and 3 on skills of candidate:
    - Resume Content: {splits}
    - Skills: {user_skills}
    - Area of Interest: {user_area_of_interest}
    """
    # Call your RAG function to generate the questions
    response = generate_rag_response(question_prompt)  # Assuming this function is defined and works
    return response.split("\n")  # Split the response into individual questions

# Fetch questions from the RAG function
questions = generate_relevant_questions()

# Interactive function to display questions and capture answers sequentially
def interactive_questionnaire(questions):
    answers = []  # Store the answers
    print("Interactive Q&A Session:\n")
    for idx, question in enumerate(questions, 1):
        question = question.strip()  # Clean up any whitespace
        if question:  # Ignore empty lines
            print(f"Question {idx}: {question}")  # Display the question
            answer = input("Your Answer: ")  # Get the user's answer
            answers.append((question, answer))  # Store the question-answer pair
            print()  # Add spacing for better readability
    return answers

# Call the interactive Q&A process
answers = interactive_questionnaire(questions)

# Display all questions and answers at the end
print("\nYour Responses:\n")
for idx, (question, answer) in enumerate(answers, 1):
    print(f"{idx}. {question}")
    print(f"   Your Answer: {answer}")


Interactive Q&A Session:

Question 1: Here are 10 interview questions for the candidate:
Your Answer: ok

Question 2: 4 based on resume and project:
Your Answer: ok

Question 3: 1. How did you engineer a robust navigation system for safe vehicle operation in adverse weather using machine learning?
Your Answer: l

Question 4: 2. Can you explain the technical details of your Hydrominex project, including the machine learning algorithms used?
Your Answer: l

Question 5: 3. How did you optimize retrieval accuracy in your AI Interviewer project to handle complex queries?
Your Answer: l

Question 6: 4. What was your role in the development of the automated news summarization and analysis tool?
Your Answer: l

Question 8: 3 based on area of interest (data science):
Your Answer: ok

Question 9: 5. What do you think are the most significant challenges in data science, and how do you stay updated with the latest developments?


KeyboardInterrupt: Interrupted by user

# Saving QA pairs in Langchain Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

# Initialize memory
memory = ConversationBufferMemory()

# Add Q&A pairs to memory
for question, answer in answers:
    memory.save_context({"question": question}, {"answer": answer})

# Access stored conversations
stored_conversations = memory.load_memory_variables({})
print("Stored Q&A:", stored_conversations)


Stored Q&A: {'history': "Human: Based on the provided context, here are 10 interview questions for the candidate:\nAI: l\nHuman: 4 questions based on resume and project:\nAI: l\nHuman: 1. How did you utilize machine learning in your Hydrominex project for safe vehicle operation in adverse weather?\nAI: k\nHuman: 2. Can you explain the technical stack used in your AI Interviewer project?\nAI: l\nHuman: 3. How did you optimize data preprocessing in your Automated News Summarization and Analysis tool?\nAI: l\nHuman: 4. What was your role in the development of the navigation system in the Hydrominex project?\nAI: \nHuman: 3 questions based on area of interest (data science):\nAI: \nHuman: 5. What inspired you to pursue data science, and how do you stay updated with the latest trends?\nAI: \nHuman: 6. Can you explain a challenging data science problem you've encountered and how you overcame it?\nAI: l\nHuman: 7. How do you think data science can be applied to real-world problems, and what c

<ipython-input-73-d684d8af7a72>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


# Judging the answers

In [ ]:
l# Function to dynamically generate questions using RAG (already exists in your previous code)
def generate_relevant_questions():
    question_prompt = f"""
    Generate 10 relevant and very particular moderate level interview questions for a candidate based on the following information.
    Generate 4 questions based on the resume and project, 3 based on the area of interest, and 3 based on the skills of the candidate:
    - Resume Content: {splits}
    - Skills: {user_skills}
    - Area of Interest: {user_area_of_interest}
    """
    # Call your RAG function to generate the questions
    response = generate_rag_response(question_prompt)  # Assuming this function is defined and works
    return response.split("\n")  # Split the response into individual questions

# Function to evaluate answers using Groq LLM
def judge_answers_groq(qa_pairs):
    scored_responses = []  # List to store scored evaluations

    for idx, (question, answer) in enumerate(qa_pairs, 1):
        # Create the judging prompt
        judging_prompt = f"""
        Evaluate the candidate's response to the following question based on relevance, clarity, and correctness.
        Strictly judge the answer for core concepts, depth of understanding, and conciseness.
        Provide a score out of 10 (whole number only) .

        Question: {question}
        Answer: {answer}

        Your evaluation should be in this format:
        - Score: <score out of 10>
        - Justification: <reasoning>
        """

        # Call Groq LLM for evaluation
        chat_completion = client.chat.completions.create(
            messages=[
                {"role": "user", "content": judging_prompt}
            ],
            model="llama-3.3-70b-versatile"
        )

        # Extract the response content
        evaluation = chat_completion.choices[0].message.content.strip()
        scored_responses.append((question, answer, evaluation))

    return scored_responses

# Generate the list of questions from the candidate's information
questions = generate_relevant_questions()

# Interactive function to display questions and capture answers sequentially
def interactive_questionnaire(questions):
    answers = []  # Store the answers
    print("Interactive Q&A Session:\n")
    for idx, question in enumerate(questions, 1):
        question = question.strip()  # Clean up any whitespace
        if question:  # Ignore empty lines
            print(f"Question {idx}: {question}")  # Display the question
            answer = input("Your Answer: ")  # Get the user's answer
            answers.append((question, answer))  # Store the question-answer pair
            print()  # Add spacing for better readability
    return answers

# Call the interactive Q&A process to get candidate's answers
answers = interactive_questionnaire(questions)

# Now, evaluate the answers using Groq LLM
judged_results = judge_answers_groq(answers)

# Display the results (scoring and justifications)
print("\nEvaluation Results:\n")
for idx, (question, answer, evaluation) in enumerate(judged_results, 1):
    print(f"Q{idx}: {question}")
    print(f"Answer: {answer}")
    print(f"Evaluation:\n{evaluation}\n")


Interactive Q&A Session:

Question 1: Here are 10 moderate-level interview questions for the candidate:
Your Answer: ok

Question 2: 4 based on the resume and project:
Your Answer: ok

Question 3: 1. How did you leverage machine learning for real-time weather forecasting in your Hydrominex project?
Your Answer: ok

Question 4: 2. Can you explain the role of sensor fusion with GPS and radar in enhancing obstacle detection?
Your Answer: l

Question 5: 3. How did you optimize retrieval accuracy to handle complex queries in your AI Interviewer project?
Your Answer: l

Question 6: 4. What techniques did you use for text preprocessing in your Automated News Summarization and Analysis tool?
Your Answer: l

Question 8: 3 based on the area of interest (data science):
Your Answer: l

Question 9: 5. How do you stay updated with the latest advancements in data science?
Your Answer: l

Question 10: 6. What do you think are the most significant challenges in data science, and how do you address them